In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
# from auxiliar_func import preprocessing

In [ ]:
df = pd.read_csv('Census-Income-KDD.csv')

# Data Preprocessing
X = df.drop('income_50k', axis=1)
y = df['income_50k']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)